In [1]:
include("../src/variationaltimedisc.jl")
using GradientRobustMultiPhysics
using ExtendableGrids
using Printf
using ExtendableSparse
using DelimitedFiles
using SimplexGridFactory
using Triangulate

In [2]:
function get_flow_data(ν, example)
    ## note that dependencies "XT" marks the function to be x- and t-dependent
    ## that causes the solver to automatically reassemble associated operators in each time step
    u = DataFunction((result, x, t) -> (
        result[1] = 3*t*(x[1]+x[2]); 
        result[2] = -3*t*(x[1]+x[2]);
        ), [2,2]; name = "u", dependencies = "XT", bonus_quadorder = 5)
    p = DataFunction((result, x) -> (
        result[1] = 0.0
        ), [1,2]; name = "p", dependencies = "X", bonus_quadorder = 5)
  
    ############## common code for all examples #####
    dt_u = eval_dt(u)
    Δu = eval_Δ(u)
    ∇u = eval_∇(u)
    ∇p = eval_∇(p)
    f = DataFunction((result, x, t) -> (
          result .= dt_u(x,t) .- ν*Δu(x,t) .+∇u(x,t)*u(x,t) .+ view(∇p(x,t),:);
        ), [2,2]; name = "f", dependencies = "XT", bonus_quadorder = 5)  
    return u, p, f
  end

get_flow_data (generic function with 1 method)

In [3]:
function run_main(;scheme = 1, ν = 1, nlevels=3, fe_type = 1, rec_type = 1, T0 = 0, Tf=1, 
    nsteps=10, reconstruct = false, example = 1, verbosity = 0)

    # set log level
  set_verbosity(verbosity)
  ## initial grid 
  #xgrid = grid_unitsquare(Triangle2D)
  # refine the grid 
  #xgrid = uniform_refine(xgrid, nlevels)  
  xgrid = simplexgrid(Triangulate;
            points=[0 0 ; 0 1 ; 1 1 ; 1 0]',
            bfaces=[1 2 ; 2 3 ; 3 4 ; 4 1 ]',
            bfaceregions=[1, 2, 3, 4],
            regionpoints=[0.5 0.5;]',
            regionnumbers=[1],
            regionvolumes=[4.0^(-nlevels-1)/2])
  println(xgrid)

  ## choose one of these (inf-sup stable) finite element type pairs
  FETypes = [H1P2{2,2}, H1P1{1}] # Taylor-Hood elements
  
  u, p, f= get_flow_data(ν, example)

  # generate FE spaces
  FES = [FESpace{FETypes[1]}(xgrid), FESpace{FETypes[2]}(xgrid)]
  # solution vector
  Solution = FEVector(["u_h", "p_h"], FES)
  res = FEVector(["u_h", "p_h"], FES)
  Solnm1 = FEVector(FES)
  interpolate!(Solnm1[1], u; time = 0.0)
  Solnm2 = FEVector(FES)
  interpolate!(Solnm2[1], u; time = 1e-15)

  ndofu = FES[1].ndofs
  ndofp = FES[2].ndofs
  # n_unknown = length(Solution.entries)  
  GradientRobustMultiPhysics.interpolate!(Solution[1], u; time = 0.)

  # mass matrix 
  M = FEMatrix{Float64}(FES)
  assemble_operator!(M[1,1], BilinearForm([Identity, Identity]))
  # velocity pressure matrices
  A = FEMatrix{Float64}(FES)
  assemble_operator!(A[1,1], LaplaceOperator(ν))
  assemble_operator!(A[1,2], LagrangeMultiplier(Divergence); At = A[2,1]) 

  rhs = FEVector{Float64}(FES)
  assemble_operator!(rhs[1], LinearForm(Identity, f); time=0.0)

  ## the convection operator is assembled to the right-hand side
  ## to keep the matrix constant in time (but we do subiterations in each timestep)
  CO = ConvectionOperator(1, Identity, 2, 2; newton = true)
  #CO_Picard = ConvectionOperator(1, Identity, 2, 2; newton = false)
  # disctrete convection operator
  DCO = GradientRobustMultiPhysics.AssemblyPattern{GradientRobustMultiPhysics.APT_NonlinearForm, Float64, ON_CELLS}(CO.name, 
        [FES[1], FES[1], FES[1]], CO.operators4arguments,CO.action,CO.apply_action_to,CO.regions)
  # Picard iteration
  #DCO = GradientRobustMultiPhysics.AssemblyPattern{GradientRobustMultiPhysics.APT_BilinearForm, Float64, ON_CELLS}(CO.name, 
  #      [FES[1], FES[1], FES[1]], CO.operators4arguments,CO.action,CO.apply_action_to,CO.regions)
  # which needs derivatives
  DCO.newton_args = CO.newton_arguments  
  # assemble the nonlinear matrix and rhs
  ANL = FEMatrix{Float64}(FES)  
  rhsNL = FEVector{Float64}(FES)
  # GradientRobustMultiPhysics.full_assemble!(ANL[1,1], rhsNL[1], DCO, [Solution[1], Solution[1]])
  dt = Array{BoundaryData,1}(undef,0)
  push!(dt, BoundaryData(BestapproxDirichletBoundary; regions = [1,2,3,4], data = u))
  dofs = boundarydata!(Solution[1], dt; time = 0.0)
  
  t0 = T0
  if scheme == 1
      tau = (Tf - T0)/nsteps
  end
  if scheme == 2
      tau = (Tf - T0)/nsteps
  end

  V1 = zeros(Float64, ndofu, 1)

  SystemMatrix = FEMatrix(FES)
  # @show SystemMatrix
  SystemRHS = FEVector(FES)
  SystemSol = FEVector(FES)
  
  step = one(Int)

  l2max = -one(Float64)
  first_step  = true
#------------------------------------------------------------------------------
  nts = 1
  oldL2 = zero(Float64); oldh1 = zero(Float64)
  eL2 = zero(Float64); eh1 = zero(Float64)
  tol = 1e-12
  max_iter = 10
  niter = zero(Int64)
  while t0 <= Tf-1e-10
    t0 = t0 + tau    
    fill!(rhs.entries, 0)
    assemble_operator!(rhs[1], LinearForm(Identity, f), time= t0 )
    V1[:, 1] = rhs[1][:]

    fill!(SystemRHS.entries, 0)
    fill!(SystemMatrix.entries.cscmatrix.nzval, 0)
    fill!(A[1,1], 0)
    if nts == 1
      if scheme == 2
        @info ("First step is with BDF1")
      end
      addblock!(SystemRHS[1], M.entries*Solution[1].entries ; factor= 1.0/tau)
      addblock!(SystemRHS[1], V1[:,1]; factor= 1.0 )

      #system matrix
      assemble_operator!(A[1, 1], LaplaceOperator(ν); time=t0 )
      addblock!(SystemMatrix[1, 1], M[1, 1]; factor= 1.0/tau)      
    else
      # system rhs
      addblock!(SystemRHS[1], M.entries*Solnm1[1].entries; factor=  2.0/tau)
      addblock!(SystemRHS[1], M.entries*Solnm2[1].entries; factor= -0.5/tau)
      addblock!(SystemRHS[1], V1[:,1]; factor= 1.0 )
      # system matrix 
      assemble_operator!(A[1, 1], LaplaceOperator(ν); time=t0 )
      addblock!(SystemMatrix[1, 1], M[1, 1]; factor= 1.5/tau)
    end
    addblock!(SystemMatrix[1, 1], A[1, 1]; factor= 1.0)
    addblock!(SystemMatrix[1, 2], A[1, 2]; factor= 1.0)
    addblock!(SystemMatrix[2, 1], A[2, 1]; factor= 1.0)

    niter = 0
    residual = 1e60
    flush!(SystemMatrix.entries)
    while residual >= tol && niter < max_iter
      fill!(ANL.entries.cscmatrix.nzval, 0)
      fill!(rhsNL.entries, 0)
      addblock!(ANL[1, 1], SystemMatrix[1, 1]; factor= 1.0)
      addblock!(ANL[1, 2], SystemMatrix[1, 2]; factor= 1.0)
      addblock!(ANL[2, 1], SystemMatrix[2, 1]; factor= 1.0)
      addblock!(rhsNL[1], SystemRHS[1]; factor= 1.0 )      
      GradientRobustMultiPhysics.full_assemble!(ANL[1,1], rhsNL[1], DCO, [Solution[1], Solution[1]])
      # Assemble only ANL for Picard iteration 
      # GradientRobustMultiPhysics.assemble!(ANL[1,1], DCO, [Solution[1], Solution[1]])
      
      dofs = boundarydata!(Solution[1], dt; time = t0)
      for dof in dofs
        rhsNL[1][dof] = 1e60 * Solution[1][dof]
        ANL[1,1][dof,dof] = 1e60
      end
      # ANL[1,1][1,1] = 1e60
      ANL[2,2][1,1] = 1e60
      # residual computation
      GradientRobustMultiPhysics.mul!(res.entries,ANL.entries,Solution.entries)
      res.entries .-= rhsNL.entries
      
      for dof in dofs 
        res[1][dof] = 0
      end
      residual = norm(res.entries)
      println(residual)
      flush!(ANL.entries)
      # solve the system 
      Solution.entries[:] = ANL.entries \ rhsNL.entries
      niter = niter + 1
    end
        
    L2Error_u = L2ErrorIntegrator(u, Identity; time= t0)
    l2 = evaluate(L2Error_u, Solution[1])

    h1_err = L2ErrorIntegrator(∇(u), Gradient; time= t0)
    h1 = evaluate(h1_err, Solution[1])

    if scheme == 2
      nts = nts + 1
      for j=1:ndofu + ndofp
        Solnm2[1][j] = Solnm1[1][j]
        Solnm1[1][j] = Solution[1][j]
      end
    end
    println(t0, "\t", sqrt(l2), '\t', sqrt(h1))
  end
  println("done")
end

run_main (generic function with 1 method)

In [4]:
function conv_test(ns, td=1, tdorder=0, ν=1, nl=2, fe_t=1, rec=1, recons=false )
  for n in ns
    run_main(scheme=1)
  end
end
conv_test([8])

ExtendableGrid{Float64, Int32};
dim: 2 nodes: 443 cells: 808 bfaces: 76


┌ Warning:  41155300 allocations during ITEMTYPE_FACE volume calculation
└ @ ExtendableGrids ~/.julia/packages/ExtendableGrids/XFxI3/src/derived.jl:937


10.397600087864987


0.007904315915832892


1.6561037565597513e-7


7.754093634603639e-10


...

5.0912794653992995e-8


3.2242164599486277e-9
4.2983666190875716e-10


1.0987304431606314e-10


3.452792074219574e-11
1.1012306842836021e-11


0.9999999999999999	2.3124653521304486e-5	0.0021042773698508868
done
